# A Crash Course in Dash

## Why Dash and what is Dash?
`Dash` is a framework for quickly building data driven web applications. It's data handling capability makes the ease of construction of data driven apps easier over a more general purpose framework such as `React` or `Angular`.

`Dash` in fact uses `React` for the front end and React components can be converted to `Dash` components. It is built upon `Flask` where the underlying `Flask` server is accessible for adding additional capabilities such as authentication or additional routes.

## Dash Basics

A `Dash` app is simply collection of _components_ whose positional relationship is governed by a _layout_ and whose interaction is governed by callbacks. Like `Flask` you specify the app by creating an `app` instance by invoking

```app = Dash(__name__)```

and like `Flask` the app is run using `run_server` with pretty much the same options as `Flask`

```
app.run_server(debug=True)
```

### Useful Dash Libraries

`Dash` has component libraries in addition to many supplied by third parties. The most basic is `dash_html_components` often imported as `html`. The next step up are `dash_core_components` often imported as `dcc`. This includes a `Graph` component which makes graphing data easy. For bootstrap fans there is `dash_bootstrap_components` imported often as `dbc`. Finally for our purpose, you'll want to use `dash_leaflet` which will be imported here as `dl`.

**Note**: In dash 2.0. `dash_html_components` is replaced by `dash.html` and `dash_core_components` is replaced by `dash.dcc`.

### Components

A dash component is simply a object on a web page. `Dash` comes with the usual HTML DOMs such as `<div>`, etc, but also can include more complex objects such as graphs and as described later `leaflet` components. In effect any `React` component can be made a dash component. Each component has properties which can be set at the creation of the component and can be modified in a callback. For example, the `html.Div` component has a `children` property which can contain text, another component or a list of components. Other important properties common to most components is the `className` property which specifies the CSS class for the component and the `style` property which specifies inline styles to be applied to the component.

### Layout

Layouts can be as complicated as any webpage can. Essentially, a layout loosely mirrors the underlying HTML for an app. For example, here is a snippet from the `Dash` documentation:
```
app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for your data.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=fig
    )
])
```
In this example, the app comprises a `div` with a major heading `h1`, another `div` and a `graph`

### Callback

Callbacks are reactive to changes in a component property and changes a property. The basic anatomy of a callback is to decorate your callback function with the `@app.callback` decorator. The decorator takes as parameters inputs, outputs and optionally state. Each of these are type `Input`, `Output` and `State` respectively of a component property. The callback is executed anytime one of the inputs changes. The result of the callback changes one or more of the outputs. A full example taken from the documentation is here:

```
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H6("Change the value in the text box to see callbacks in action!"),
    html.Div([
        "Input: ",
        dcc.Input(id='my-input', value='initial value', type='text')
    ]),
    html.Br(),
    html.Div(id='my-output'),

])


@app.callback(
    Output(component_id='my-output', component_property='children'),
    Input(component_id='my-input', component_property='value')
)
def update_output_div(input_value):
    return 'Output: {}'.format(input_value)


if __name__ == '__main__':
    app.run_server(debug=True)
```

In this example, every time the input box `my-input` changes, the content of the `div` `my-output`. Because the callbacks require inputs, states and outputs to be positional it cam make maintenance difficult. They have recently introduced "flexible callback signatures" which make organization easier however, I prefer the plugin `dict_callback`. Available at https://github.com/WestHealth/dash-dict-callback.

#### Idiosyncracies

Because of the way `Dash` organizes callbacks as graphs, they strive to avoid race conditions in doing so, they only allow each property of each component be the output of one callback. This often creates requires you to create mega callback functions with lots of mega callbacks. Managing this is where `dict_callback` can be useful.



## For this workshop

You may run this code separately as a python app. However, since the format of this workshop is in a Jupyter notebook, JupyterDash is used. In order to use it, you will need to install `jupyter_dash`. If you run the notebook in a docker container as I do, you will also need to install `jupyter-server-proxy` which proxies the `Dash` app to use the same port as the notebook.

The main two differences between using `JupyterDash` versus `Dash` is you create it using `JupyterDash` rather than `Dash` and the `run_server` method has an extra `mode` paramenter that can be set to `inline`.

I also prefer to randomly assign the port to the app so that I can embed many apps in the same notebook or notebook container.

In [ ]:
import dash
import dash.dcc as dcc
import dash.html as html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import random

#### This next line is a hack. For some reason the JupyterDash.infer_jupyter_proxy_config doesn't necessarily work with this line in a separate cell

In [ ]:
from jupyter_dash.comms import _send_jupyter_config_comm_request
_send_jupyter_config_comm_request()

In [ ]:
JupyterDash.infer_jupyter_proxy_config()

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H6("Change the value in the text box to see callbacks in action!"),
    html.Div([
        "Input: ",
        dcc.Input(id='my-input', value='initial value', type='text')
    ]),
    html.Br(),
    html.Div(id='my-output'),
])


@app.callback(Output(component_id='my-output', component_property='children'),
              Input(component_id='my-input', component_property='value'))
def update_output_div(input_value):
    return 'Output: {}'.format(input_value)


if __name__ == '__main__':
    app.run_server(mode='inline', port=random.choice(range(2000, 10000)))

In [ ]:
NOTEBOOK_PORT = 2000
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H6("Change the value in the text box to see callbacks in action!"),
    html.Div([
        "Input: ",
        dcc.Input(id='my-input', value='initial value', type='text')
    ]),
    html.Br(),
    html.Div(id='my-output'),
])


@app.callback(Output(component_id='my-output', component_property='children'),
              Input(component_id='my-input', component_property='value'))
def update_output_div(input_value):
    return 'Output: {}'.format(input_value)


if __name__ == '__main__':
    app.run_server(mode='inline', port=NOTEBOOK_PORT)

In [ ]:
app._terminate_server_for_port("localhost", NOTEBOOK_PORT)